In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os.path
from os import path
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras import backend as K
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.preprocessing import OneHotEncoder

In [ ]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data1_train = pd.read_csv("/content/drive/MyDrive/Data mưa/cv1_data_train_92022.csv")
data1_test = pd.read_csv("/content/drive/MyDrive/Data mưa/cv1_data_test_92022.csv")
data2_train = pd.read_csv("/content/drive/MyDrive/Data mưa/cv2_data_train_92022.csv")
data2_test = pd.read_csv('/content/drive/MyDrive/Data mưa/cv2_data_test_92022.csv')
data3_train = pd.read_csv('/content/drive/MyDrive/Data mưa/cv3_data_train_92022.csv')
data3_test = pd.read_csv('/content/drive/MyDrive/Data mưa/cv3_data_test_92022.csv')
data4_train = pd.read_csv('/content/drive/MyDrive/Data mưa/cv4_data_train_92022.csv')
data4_test = pd.read_csv('/content/drive/MyDrive/Data mưa/cv4_data_test_92022.csv')
data5_train = pd.read_csv('/content/drive/MyDrive/Data mưa/cv5_data_train_92022.csv')
data5_test = pd.read_csv('/content/drive/MyDrive/Data mưa/cv5_data_test_92022.csv')
data6_train = pd.read_csv('/content/drive/MyDrive/Data mưa/cv6_data_train_92022.csv')
data6_test = pd.read_csv('/content/drive/MyDrive/Data mưa/cv6_data_test_92022.csv')
data7_train = pd.read_csv('/content/drive/MyDrive/Data mưa/cv7_data_train_92022.csv')
data7_test = pd.read_csv('/content/drive/MyDrive/Data mưa/cv7_data_test_92022.csv')
data8_train = pd.read_csv('/content/drive/MyDrive/Data mưa/cv8_data_train_92022.csv')
data8_test = pd.read_csv('/content/drive/MyDrive/Data mưa/cv8_data_test_92022.csv')
data9_train = pd.read_csv('/content/drive/MyDrive/Data mưa/cv9_data_train_92022.csv')
data9_test = pd.read_csv('/content/drive/MyDrive/Data mưa/cv9_data_test_92022.csv')
data10_train = pd.read_csv('/content/drive/MyDrive/Data mưa/cv10_data_train_92022.csv')
data10_test = pd.read_csv('/content/drive/MyDrive/Data mưa/cv10_data_test_92022.csv')
data11_train = pd.read_csv('/content/drive/MyDrive/Data mưa/cv11_data_train_92022.csv')
data11_test = pd.read_csv('/content/drive/MyDrive/Data mưa/cv11_data_test_92022.csv')
data12_train = pd.read_csv('/content/drive/MyDrive/Data mưa/cv12_data_train_92022.csv')
data12_test = pd.read_csv('/content/drive/MyDrive/Data mưa/cv12_data_test_92022.csv')
data13_train = pd.read_csv('/content/drive/MyDrive/Data mưa/cv13_data_train_92022.csv')
data13_test = pd.read_csv('/content/drive/MyDrive/Data mưa/cv13_data_test_92022.csv')



In [ ]:
data_train = data11_train
data_test = data11_test

In [ ]:
# data_train = pd.concat([data1_train, data1_test, data3_train, data3_test, data4_train, data4_test, data5_train, data5_test,
#                        data6_train, data6_test, data7_train, data7_test, data8_train, data8_test, data9_train, data9_test,
#                        data10_train, data10_test, data11_train, data11_test, data12_train, data12_test, data13_train, data13_test])
# data_test = pd.concat([data2_train, data2_test])

In [ ]:
data_train = data_train.drop(columns = ['x_id', 'y_id', 'DEM_value', 'cloud_value'])


In [ ]:

data_test = data_test.drop(columns = ['x_id', 'y_id', 'DEM_value', 'cloud_value'])


In [ ]:
data_train['utc_time'] = pd.to_datetime(data_train['utc_time'])
data_train['hour'] = data_train['utc_time'].apply(lambda x: x.hour)
import datetime

def function(x):
    x = x.timetuple()
    jdate = x.tm_yday
    return(jdate)
data_train['date'] = data_train['utc_time'].apply(lambda x: function(x))


In [ ]:
data_test['utc_time'] = pd.to_datetime(data_test['utc_time'])
data_test['hour'] = data_test['utc_time'].apply(lambda x: x.hour)
import datetime

def function(x):
    x = x.timetuple()
    jdate = x.tm_yday
    return(jdate)
data_test['date'] = data_test['utc_time'].apply(lambda x: function(x))

In [ ]:


data_train['day'] = data_train['utc_time'].apply(lambda x: x.day)
data_train['month'] = data_train['utc_time'].apply(lambda x: x.month)

In [ ]:
data_test['day'] = data_test['utc_time'].apply(lambda x: x.day)
data_test['month'] = data_test['utc_time'].apply(lambda x: x.month)

In [ ]:
def split_day(x):
  string = '{0:05b}'.format(x)
  arr = [*string]
  arr = [int(a) for a in arr]
  return arr
def split_month(x):
  string = '{0:04b}'.format(x)
  arr = [*string]
  arr = [int(a) for a in arr]
  return arr
def fill_columns(data_test):
  data_test['bit_day1'] = data_test['utc_time'].apply(lambda x: split_day(x.day)[0])
  data_test['bit_day2'] = data_test['utc_time'].apply(lambda x: split_day(x.day)[1])
  data_test['bit_day3'] = data_test['utc_time'].apply(lambda x: split_day(x.day)[2])
  data_test['bit_day4'] = data_test['utc_time'].apply(lambda x: split_day(x.day)[3])
  data_test['bit_day5'] = data_test['utc_time'].apply(lambda x: split_day(x.day)[4])
  data_test['bit_month1'] = data_test['utc_time'].apply(lambda x: split_day(x.month)[1])
  data_test['bit_month2'] = data_test['utc_time'].apply(lambda x: split_day(x.month)[2])
  data_test['bit_month3'] = data_test['utc_time'].apply(lambda x: split_day(x.month)[3])
  data_test['bit_month4'] = data_test['utc_time'].apply(lambda x: split_day(x.month)[4])

In [ ]:
fill_columns(data_test)
fill_columns(data_train)

In [ ]:
data_train = data_train.drop(columns = 'utc_time')

In [ ]:
data_test = data_test.drop(columns = 'utc_time')

In [ ]:
data_train 

,Name,id,lon,lat,value,irb_value,wvb_b09,b09_b10,b10_b16,b11_irb,...,month,bit_day1,bit_day2,bit_day3,bit_day4,bit_day5,bit_month1,bit_month2,bit_month3,bit_month4
0,226708,Chiềng Bôm,103.6502,21.42961,3.0,288.15143,-7.69904,-4.80994,-13.21002,-3.29143,...,9,0,1,1,1,0,1,0,0,1
1,692253,HienChung,104.8681,20.42085,0.2,294.00000,-7.01337,-9.26092,-20.46154,-3.82250,...,8,0,0,0,1,0,1,0,0,0
2,854,DapDangThoDo,108.7936,13.92894,0.0,270.71997,-11.72998,-9.25671,-3.26965,-1.99664,...,10,1,1,1,1,1,1,0,1,0
3,79361,KienThiet,105.3138,21.99546,0.0,263.11273,-10.94996,-6.19879,-2.67926,-1.19269,...,11,0,1,1,1,1,1,0,1,1
4,92261,TruongXuan,107.6333,12.10000,0.2,265.90723,-9.17822,-5.08417,-4.94671,-2.21363,...,9,1,0,0,1,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
457266,91828,PhuocTan,108.8086,11.85611,0.2,267.00000,-7.08404,-5.55664,-3.86242,-2.15723,...,9,0,0,0,1,1,1,0,0,1
457267,93006,LoKu,108.5122,14.15417,0.0,275.27075,-10.88885,-7.00350,-1.49944,-1.88225,...,11,0,1,1,1,1,1,0,1,1
457268,924774,Trà Giáp,108.2542,15.22222,0.0,264.80000,-9.24500,-8.70629,-6.13874,-1.69453,...,8,0,0,1,0,1,1,0,0,0
457269,3403,CuPrao,108.6633,12.86814,0.0,277.70850,-8.21887,-9.90545,-10.16254,-0.91310,...,8,0,0,1,1,0,1,0,0,0


In [ ]:
data_train = data_train.sort_values(by = ['id', 'date', 'hour'])


In [ ]:
data_test = data_test.sort_values(by = ['id', 'date', 'hour'])

In [ ]:
data_train

,Name,id,lon,lat,value,irb_value,wvb_b09,b09_b10,b10_b16,b11_irb,...,month,bit_day1,bit_day2,bit_day3,bit_day4,bit_day5,bit_month1,bit_month2,bit_month3,bit_month4
318796,356642,A Bung,107.0261,16.36830,0.0,247.32220,-6.96283,-3.32501,-1.65502,-0.45721,...,7,1,0,1,0,0,0,1,1,1
289590,356642,A Bung,107.0261,16.36830,0.0,261.96362,-8.14667,-7.26477,-5.58856,-1.25964,...,7,1,0,1,0,0,0,1,1,1
143659,356642,A Bung,107.0261,16.36830,1.0,265.90723,-7.87616,-8.39807,-7.45459,-1.06446,...,7,1,0,1,0,0,0,1,1,1
36050,356642,A Bung,107.0261,16.36830,0.0,219.19500,-0.28663,-0.15832,0.38354,-0.70098,...,7,1,0,1,0,0,0,1,1,1
69852,356642,A Bung,107.0261,16.36830,0.0,234.04285,-2.94098,-2.44903,0.04150,-0.70453,...,7,1,0,1,0,0,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88321,299387,Đức Vân - Bắc Kạn,106.0331,22.44722,0.0,259.01727,-6.88428,-3.62695,-1.15674,-2.77727,...,9,1,0,0,1,1,1,0,0,1
215304,299387,Đức Vân - Bắc Kạn,106.0331,22.44722,0.0,257.19300,-6.88428,-1.94995,-1.54572,-2.29298,...,9,1,0,0,1,1,1,0,0,1
34857,299387,Đức Vân - Bắc Kạn,106.0331,22.44722,0.2,252.74902,-6.38220,-3.31433,-0.24457,-1.34900,...,9,1,0,0,1,1,1,0,0,1
324779,299387,Đức Vân - Bắc Kạn,106.0331,22.44722,0.2,254.68500,-6.28107,-2.16149,-1.33851,-2.56500,...,9,1,0,0,1,1,1,0,0,1


In [ ]:
data_test

,Name,id,lon,lat,value,irb_value,wvb_b09,b09_b10,b10_b16,b11_irb,...,month,bit_day1,bit_day2,bit_day3,bit_day4,bit_day5,bit_month1,bit_month2,bit_month3,bit_month4
54836,356642,A Bung,107.0261,16.36830,0.0,243.69000,-5.65088,-2.40198,-1.00445,-0.91998,...,10,1,0,0,1,1,1,0,1,0
1270,356642,A Bung,107.0261,16.36830,0.0,227.60858,-1.76996,0.62750,-0.52002,-1.03692,...,10,1,0,0,1,1,1,0,1,0
36764,356642,A Bung,107.0261,16.36830,0.0,233.16858,-3.38746,-0.40850,0.42596,-0.88855,...,10,1,0,0,1,1,1,0,1,0
52117,356642,A Bung,107.0261,16.36830,0.0,235.76251,-2.87884,-1.56202,0.67426,-0.38678,...,10,1,0,0,1,1,1,0,1,0
27316,356642,A Bung,107.0261,16.36830,0.0,231.38000,-1.83997,-0.58001,0.67498,-1.29168,...,10,1,0,0,1,1,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49237,630250,Đức Thông,106.2797,22.47917,0.0,284.19714,-10.38653,-9.75569,-9.89727,-2.36284,...,10,1,0,1,0,0,1,0,1,0
30501,630250,Đức Thông,106.2797,22.47917,0.0,281.47302,-10.16999,-9.25671,-9.66516,-2.79762,...,10,1,0,1,0,0,1,0,1,0
62346,630250,Đức Thông,106.2797,22.47917,0.0,263.68000,-9.50000,-5.95428,-4.72571,-1.75996,...,10,1,0,1,0,0,1,0,1,0
18569,630250,Đức Thông,106.2797,22.47917,0.0,263.68000,-9.50000,-5.95428,-4.72571,-1.75996,...,10,1,0,1,0,0,1,0,1,0


In [ ]:
data_train = data_train.reset_index(drop = True)

In [ ]:
data_test = data_test.reset_index(drop = True)

In [ ]:
data_train['rain'] = data_train['value'].apply(lambda x: 0 if x == 0 else 1)

In [ ]:
data_test['rain'] = data_test['value'].apply(lambda x: 0 if x == 0 else 1)

In [ ]:
data_train['rain'].value_counts()

0    274360
1    182911
Name: rain, dtype: int64

In [ ]:
data_train.shape

(457271, 25)

In [ ]:
data_test.shape

(75667, 25)

In [ ]:
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# data_train[['irb_value']] = scaler.fit_transform(data_train[['irb_value']])
# data_train[['b10_b16']] = scaler.fit_transform(data_train[['b10_b16']])
# data_train[['b11_irb']] = scaler.fit_transform(data_train[['b11_irb']])
# data_train[['irb_i2b']] = scaler.fit_transform(data_train[['irb_i2b']])
# data_train[['wvb_b09']] = scaler.fit_transform(data_train[['wvb_b09']])
# data_train[['b09_b10']] = scaler.fit_transform(data_train[['b09_b10']])
# data_train[['lon']] = scaler.fit_transform(data_train[['lon']])
# data_train[['lat']] = scaler.fit_transform(data_train[['lat']])
# data_train[['hour']] = scaler.fit_transform(data_train[['hour']])
# data_train[['date']] = scaler.fit_transform(data_train[['date']])

In [ ]:
# data_test[['irb_value']] = scaler.fit_transform(data_test[['irb_value']])
# data_test[['b10_b16']] = scaler.fit_transform(data_test[['b10_b16']])
# data_test[['b11_irb']] = scaler.fit_transform(data_test[['b11_irb']])
# data_test[['irb_i2b']] = scaler.fit_transform(data_test[['irb_i2b']])
# data_test[['wvb_b09']] = scaler.fit_transform(data_test[['wvb_b09']])
# data_test[['b09_b10']] = scaler.fit_transform(data_test[['b09_b10']])
# data_test[['lon']] = scaler.fit_transform(data_test[['lon']])
# data_test[['lat']] = scaler.fit_transform(data_test[['lat']])
# data_test[['hour']] = scaler.fit_transform(data_test[['hour']])
# data_test[['date']] = scaler.fit_transform(data_test[['date']])

In [ ]:

input_train = []
output_train = []
no_of_sample = len(data_train)
WINDOW_SIZE = 3 # 3 gio
for i in range(no_of_sample - WINDOW_SIZE + 1):
  next_idx = i + WINDOW_SIZE - 1
  if(data_train['hour'][next_idx] == (data_train['hour'][i] + WINDOW_SIZE - 1) % 24 and data_train['hour'][next_idx - 1] == (data_train['hour'][i] + WINDOW_SIZE - 2) and data_train['lat'][next_idx] == data_train['lat'][i] ):
    X = data_train.loc[i:i + WINDOW_SIZE - 1]
    X = X.drop(columns = ['hour', 'lat', 'value','lon', 'date', 'Name', 'id','rain'])
    X.to_numpy()
    input_train.append(X)
    output_train.append(data_train['rain'][i+WINDOW_SIZE - 1])



In [ ]:
input_test = []
output_test = []
no_of_sample = len(data_test)
WINDOW_SIZE = 3 # 3 gio
for i in range(no_of_sample - WINDOW_SIZE + 1):
  next_idx = i + WINDOW_SIZE - 1
  if(data_test['hour'][next_idx] == (data_test['hour'][i] + WINDOW_SIZE - 1) % 24 and data_test['hour'][next_idx - 1] == (data_test['hour'][i] + WINDOW_SIZE - 2) and data_test['lat'][next_idx] == data_test['lat'][i] ):
    X = data_test.loc[i:i + WINDOW_SIZE - 1]
    X = X.drop(columns = ['hour', 'lat', 'value','lon', 'date', 'Name', 'id','rain'])
    X.to_numpy()
    input_test.append(X)
    output_test.append(data_test['rain'][i+WINDOW_SIZE - 1])

In [ ]:
X_train = np.array(input_train)

In [ ]:
y_train = np.array(output_train)

In [ ]:
X_test = np.array(input_test)

In [ ]:
y_test = np.array(output_test)

In [ ]:
X_train.shape

(167304, 3, 17)

In [ ]:
X_test.shape

(21005, 3, 17)

In [ ]:
#model
regressor = Sequential()
regressor.add(LSTM(units = 50, input_shape = (X_train.shape[1], X_train.shape[2])))
regressor.add(Dense(units = 1, activation = 'sigmoid'))
regressor.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=['acc', f1_m])

In [ ]:
regressor.fit(X_train, y_train, epochs = 32, batch_size= 512 )

Epoch 1/32
327/327 [==============================] - 7s 14ms/step - loss: 0.6869 - acc: 0.5437 - f1_m: 0.6710
Epoch 2/32
327/327 [==============================] - 5s 14ms/step - loss: 0.6820 - acc: 0.5608 - f1_m: 0.6618
Epoch 3/32
327/327 [==============================] - 5s 14ms/step - loss: 0.6771 - acc: 0.5752 - f1_m: 0.6601
Epoch 4/32
327/327 [==============================] - 5s 16ms/step - loss: 0.6729 - acc: 0.5846 - f1_m: 0.6603
Epoch 5/32
327/327 [==============================] - 5s 15ms/step - loss: 0.6689 - acc: 0.5937 - f1_m: 0.6623
Epoch 6/32
327/327 [==============================] - 5s 15ms/step - loss: 0.6654 - acc: 0.6012 - f1_m: 0.6655
Epoch 7/32
327/327 [==============================] - 5s 14ms/step - loss: 0.6624 - acc: 0.6082 - f1_m: 0.6687
Epoch 8/32
327/327 [==============================] - 5s 15ms/step - loss: 0.6604 - acc: 0.6117 - f1_m: 0.6684
Epoch 9/32
327/327 [==============================] - 5s 14ms/step - loss: 0.6581 - acc: 0.6154 - f1_m: 0.6716
E

In [ ]:
y_predict = regressor.predict(X_test).flatten()

In [ ]:
y_predict.shape

(21005,)

In [ ]:
y_predict[0:5]

array([0.5941191 , 0.57558554, 0.6375683 , 0.7925192 , 0.76727176],
      dtype=float32)

In [ ]:
y_train_predict = regressor.predict(X_train).flatten()

In [ ]:
train_predict = [0 if i < 0.5 else 1 for i in y_train_predict]

In [ ]:
test_predict = [0 if i < 0.5 else 1 for i in y_predict]

In [ ]:
unique, counts = np.unique(np.array(test_predict), return_counts=True)

np.asarray((unique, counts)).T

array([[    0,  8023],
       [    1, 12982]])

In [ ]:
unique1, counts1 = np.unique(np.array(y_test), return_counts=True)

np.asarray((unique1, counts1)).T

array([[    0,  9278],
       [    1, 11727]])

In [ ]:
train_tn, train_fp, train_fn, train_tp = confusion_matrix(y_train, train_predict).ravel()

prec_no_rain = train_tn / (train_tn + train_fn)
prec_rain = train_tp / (train_tp + train_fp)

recall_no_rain = train_tn / (train_tn + train_fp)
recall_rain = train_tp / (train_tp + train_fn)

print('precision no rain {:.2f}'.format(prec_no_rain))
print('precision rain: {:.2f}'.format(prec_rain))

print('recall no rain: {:.2f}'.format(recall_no_rain))
print('recall rain: {:.2f}'.format(recall_rain))

print('f1 no rain: {:.2f}'.format(2 * (prec_no_rain * recall_no_rain) / (prec_no_rain + recall_no_rain)))
print('f1 rain: {:.2f}'.format(2 * (prec_rain * recall_rain) / (prec_rain + recall_rain)))

print('accuracy: {:.2f}'.format(accuracy_score(y_train, train_predict)))

precision no rain 0.62
precision rain: 0.65
recall no rain: 0.58
recall rain: 0.68
f1 no rain: 0.60
f1 rain: 0.67
accuracy: 0.64


In [ ]:
test_tn, test_fp, test_fn, test_tp = confusion_matrix(y_test, test_predict).ravel()

prec_no_rain = test_tn / (test_tn + test_fn)
prec_rain = test_tp / (test_tp + test_fp)

recall_no_rain = test_tn / (test_tn + test_fp)
recall_rain = test_tp / (test_tp + test_fn)

print('precision no rain {:.2f}'.format(prec_no_rain))
print('precision rain: {:.2f}'.format(prec_rain))

print('recall no rain: {:.2f}'.format(recall_no_rain))
print('recall rain: {:.2f}'.format(recall_rain))

print('f1 no rain: {:.2f}'.format(2 * (prec_no_rain * recall_no_rain) / (prec_no_rain + recall_no_rain)))
print('f1 rain: {:.2f}'.format(2 * (prec_rain * recall_rain) / (prec_rain + recall_rain)))

print('accuracy: {:.2f}'.format(accuracy_score(y_test, test_predict)))

precision no rain 0.48
precision rain: 0.58
recall no rain: 0.41
recall rain: 0.64
f1 no rain: 0.44
f1 rain: 0.61
accuracy: 0.54
